### What-If Tool Demo toxicity text model comparison

This notebook shows use of the [What-If Tool](https://pair-code.github.io/what-if-tool) to compare two text models that determine sentence toxicity, one of which has had some debiasing performed during training.

This notebook loads two pretrained toxicity models from [ConversationAI](https://github.com/conversationai/unintended-ml-bias-analysis) and compares them on the [wikipedia comments dataset](https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973).

This notebook also shows how the What-If Tool can be used on non-TensorFlow models. In this case, these models are keras models that do not use tensorflow Examples as an input format. These models can be analyzed in the What-If Tool by supplying a custom prediction function to WitWidget.

It also shows use of a user-provided custom distance function (for counterfactual analysis and datapoint similarity visualizations). The [tf.Hub Universal Sentence Encoder](https://tfhub.dev/google/universal-sentence-encoder/2) is used to compute the similarity of the input text comments.


In [2]:
#@title Install the What-If Tool widget if running in colab {display-mode: "form"}

# If running in colab then pip install, otherwise no need.
try:
    import google.colab
    !pip install --upgrade tensorflow>=2.0.0 witwidget # type: ignore
except Exception:
    !pip install witwidget
    !jupyter nbextension install --py --symlink --sys-prefix witwidget
    !jupyter nbextension enable --py --sys-prefix witwidget

In [15]:
#@title Download the pretrained keras model files
!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/cnn_wiki_tox_v3_model.h5 -o ./cnn_wiki_tox_v3_model.h5
!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/cnn_wiki_tox_v3_hparams.h5 -o ./cnn_wiki_tox_v3_hparams.h5
!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/cnn_wiki_tox_v3_tokenizer.pkl -o ./cnn_wiki_tox_v3_tokenizer.pkl

!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/cnn_debias_tox_v3_model.h5 -o ./cnn_debias_tox_v3_model.h5
!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/cnn_debias_tox_v3_hparams.h5 -o ./cnn_debias_tox_v3_hparams.h5
!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/cnn_debias_tox_v3_tokenizer.pkl -o ./cnn_debias_tox_v3_tokenizer.pkl

!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/wiki_test.csv -o ./wiki_test.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 59.5M  100 59.5M    0     0  18.9M      0  0:00:03  0:00:03 --:--:-- 18.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   227  100   227    0     0   1759      0 --:--:-- --:--:-- --:--:--  1759
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.7M  100 10.7M    0     0  13.2M      0 --:--:-- --:--:-- --:--:-- 13.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61.6M  100 61.6M    0     0  20.1M      0  0:00:03  0:00:03 --:--:-- 20.1M
  % Total    % Received % Xferd  Average Speed   Tim

In [3]:
#@title Load the keras models
import sys
import tensorflow as tf
from six.moves import cPickle as pkl

def pkl_load(f):
  return pkl.load(f) if sys.version_info < (3, 0) else pkl.load(
    f, encoding='latin1')

model1 = tf.keras.models.load_model('cnn_wiki_tox_v3_model.h5')
with open('cnn_wiki_tox_v3_tokenizer.pkl', 'rb') as f:
  tokenizer1 = pkl_load(f)
tokenizer1.oov_token = None # quick fix for version issues

model2 = tf.keras.models.load_model('cnn_debias_tox_v3_model.h5')
with open('cnn_debias_tox_v3_tokenizer.pkl', 'rb') as f:
  tokenizer2 = pkl_load(f)
tokenizer2.oov_token = None # quick fix for version issues

2022-10-09 20:14:27.044433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
#@title Define custom prediction functions so that WIT infers using keras models
import tensorflow as tf

# Set up model helper functions:
PADDING_LEN = 250

# Convert list of tf.Examples to list of comment strings.
def examples_to_strings(examples):
  texts = [ex.features.feature['comment'].bytes_list.value[0] for ex in examples]
  if sys.version_info >= (3, 0):
    texts = [t.decode('utf-8') for t in texts]
  return texts

# Get raw string out of tf.Example and prepare it for keras model input
def examples_to_model_in(examples, tokenizer):
  texts = examples_to_strings(examples)
  # Tokenize string into fixed length sequence of integer based on tokenizer 
  # and model padding
  text_sequences = tokenizer.texts_to_sequences(texts)
  model_ins = tf.keras.preprocessing.sequence.pad_sequences(
      text_sequences, maxlen=PADDING_LEN)
  return model_ins

# WIT predict functions:
def custom_predict_1(examples_to_infer):
  model_ins = examples_to_model_in(examples_to_infer, tokenizer1)
  preds = model1.predict(model_ins)
  return preds

def custom_predict_2(examples_to_infer):
  model_ins = examples_to_model_in(examples_to_infer, tokenizer2)
  preds = model2.predict(model_ins)
  return preds

In [6]:
#@title Define helper functions for dataset conversion from csv to tf.Examples
import numpy as np
import tensorflow as tf

# Converts a dataframe into a list of tf.Example protos.
def df_to_examples(df, columns=None):
  examples = []
  if columns == None:
    columns = df.columns.values.tolist()
  for index, row in df.iterrows():
    example = tf.train.Example()
    for col in columns:
      if df[col].dtype is np.dtype(np.int64):
        example.features.feature[col].int64_list.value.append(int(row[col]))
      elif df[col].dtype is np.dtype(np.float64):
        example.features.feature[col].float_list.value.append(row[col])
      elif row[col] == row[col]:
        example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
    examples.append(example)
  return examples

# Converts a dataframe column into a column of 0's and 1's based on the provided test.
# Used to force label columns to be numeric for binary classification using a TF estimator.
def make_label_column_numeric(df, label_column, test):
  df[label_column] = np.where(test(df[label_column]), 1, 0)

In [7]:
#@title Read the dataset from CSV and process it for model {display-mode: "form"}
import pandas as pd

# Set the path to the CSV containing the dataset to train on.
csv_path = 'wiki_test.csv'

# Set the column names for the columns in the CSV. If the CSV's first line is a header line containing
# the column names, then set this to None.
csv_columns = None

# Read the dataset from the provided CSV and print out information about it.
df = pd.read_csv(csv_path, names=csv_columns, skipinitialspace=True)
df = df[['is_toxic', 'comment']]

# Remove non ascii characters
comments = df['comment'].values
proc_comments = []
comment_lengths = []
for c in comments:
  try:
    if sys.version_info >= (3, 0):
      c = bytes(c, 'utf-8')
    c = c.decode('unicode_escape')
    if sys.version_info < (3, 0):
      c = c.encode('ascii', 'ignore')
    proc_comment = c.strip()
  except:
    proc_comment = ''
  proc_comments.append(proc_comment)
  comment_lengths.append(len(proc_comment.split()))

df = df.assign(comment=proc_comments)
df['comment length'] = comment_lengths

label_column = 'is_toxic'
make_label_column_numeric(df, label_column, lambda val: val)

examples = df_to_examples(df)

/var/folders/11/znbx5sdn46zb_k31yz8gfrd00000gn/T/ipykernel_14484/3034194029.py:23: DeprecationWarning: invalid escape sequence '\ '
  c = c.decode('unicode_escape')
/var/folders/11/znbx5sdn46zb_k31yz8gfrd00000gn/T/ipykernel_14484/3034194029.py:23: DeprecationWarning: invalid escape sequence '\e'
  c = c.decode('unicode_escape')
/var/folders/11/znbx5sdn46zb_k31yz8gfrd00000gn/T/ipykernel_14484/3034194029.py:23: DeprecationWarning: invalid escape sequence '\.'
  c = c.decode('unicode_escape')
/var/folders/11/znbx5sdn46zb_k31yz8gfrd00000gn/T/ipykernel_14484/3034194029.py:23: DeprecationWarning: invalid escape sequence '\A'
  c = c.decode('unicode_escape')
/var/folders/11/znbx5sdn46zb_k31yz8gfrd00000gn/T/ipykernel_14484/3034194029.py:23: DeprecationWarning: invalid escape sequence '\,'
  c = c.decode('unicode_escape')
/var/folders/11/znbx5sdn46zb_k31yz8gfrd00000gn/T/ipykernel_14484/3034194029.py:23: DeprecationWarning: invalid escape sequence '\_'
  c = c.decode('unicode_escape')
/var/folde

In [8]:
#@title Display descriptive stats of data {display-mode: "form"}
df.describe()

,is_toxic,comment length
count,31866.000000,31866.000000
mean,0.095651,68.168644
std,0.294116,99.563199
min,0.000000,0.000000
25%,0.000000,17.000000
50%,0.000000,37.000000
75%,0.000000,76.000000
max,1.000000,1403.000000


In [9]:
#@title Display format  {display-mode: "form"}
print(examples[:10])

[features {
  feature {
    key: "comment"
    value {
      bytes_list {
        value: "Elected or Electoral? JHK"
      }
    }
  }
  feature {
    key: "comment length"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "is_toxic"
    value {
      int64_list {
        value: 0
      }
    }
  }
}
, features {
  feature {
    key: "comment"
    value {
      bytes_list {
        value: "Please relate the ozone hole to increases in cancer, and provide figures. Otherwise, this article will be biased toward the environmentalist anti-CFC point of view instead of being neutral. Ed Poor"
      }
    }
  }
  feature {
    key: "comment length"
    value {
      int64_list {
        value: 31
      }
    }
  }
  feature {
    key: "is_toxic"
    value {
      int64_list {
        value: 0
      }
    }
  }
}
, features {
  feature {
    key: "comment"
    value {
      bytes_list {
        value: "`    I\'m not sure if it\'s properly called ``fifth tunin

In [10]:
#@title Define a custom distance function for comparing datapoints (uses tf.Hub) {display-mode: "form"}

import tensorflow as tf
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# For this use-case, we set distance between datapoints to be cosine distance
# between unit-normalized embeddings of each datapoint from the tf.Hub
# Universal Sentence Encoder.
def universal_sentence_encoder_distance(input_example, examples_to_compare, _):
  # Extract comment strings
  input_sentence = examples_to_strings([input_example])[0]
  sentences = examples_to_strings(examples_to_compare)

  # Normalize all embeddings for cosine distance operation
  input_emb = tf.squeeze(tf.nn.l2_normalize(embed([input_sentence]), axis=1))
  sentences_emb = tf.nn.l2_normalize(embed(sentences), axis=1)

  # Tile the input example for easy comparison to all examples
  multiply = tf.constant([len(examples_to_compare)])
  input_matrix = tf.reshape(tf.tile(input_emb, multiply),
                            [multiply[0], tf.shape(input_emb)[0]])
  
  # Compute cosine distance from input example to all examples.
  cosine_distance = tf.keras.losses.CosineSimilarity(
      axis=1, reduction=tf.losses.Reduction.NONE)
  distances = cosine_distance(sentences_emb, input_matrix)
  results = tf.squeeze(distances)
  return results.numpy().tolist()

In [11]:
#@title Invoke What-If Tool for the data and two models (Note that this step may take a while due to prediction speed of the toxicity model){display-mode: "form"}
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder
num_datapoints = 1000  #@param {type: "number"}
tool_height_in_px = 720  #@param {type: "number"}

# Setup the tool with the test examples and the trained classifier
config_builder = WitConfigBuilder(examples[:num_datapoints]).set_custom_predict_fn(
  custom_predict_1).set_compare_custom_predict_fn(custom_predict_2).set_custom_distance_fn(
      universal_sentence_encoder_distance)

wv = WitWidget(config_builder, height=tool_height_in_px)

In [12]:
wv

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'are_sequence_examples': False, 'inferenc…